In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import seaborn as sns
from matplotlib.colors import ListedColormap
from scipy.interpolate import interp1d

%matplotlib inline

In [14]:
df = pd.read_csv('pseudo_sigmoid_batches_long.csv')

In [15]:
df.columns


Index(['batch', 'data_point', 'value'], dtype='object')

In [16]:
df = df.rename(columns = {'value':'Signal', 'data_point':'Volume', 'batch':'Batch'})
df["Campaign"] = "C1"



In [17]:
df = df[df['Batch'] == 'Batch_1']

In [18]:
# Normalization as described in the "Moment Analysis" section
def normalize_signal(signal):
    return (signal - signal.min()) / (signal.max() - signal.min())




In [19]:
def calculate_direct_af(volume, signal):
    # Sort the data by Volume
    sorted_indices = np.argsort(volume)
    volume = volume[sorted_indices]
    signal = signal[sorted_indices]

    # Normalize the signal
    normalized_signal = normalize_signal(signal)

    # Create interpolation function
    interp_func = interp1d(normalized_signal, volume, kind='linear', bounds_error=False, fill_value="extrapolate")

    thresholds_low = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
    thresholds_high = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70]
    cv_mid = interp_func(0.5)

    ratios = []
    for low, high in zip(thresholds_low, thresholds_high):
        cv_a = interp_func(low)
        cv_b = interp_func(high)
        ratios.append((cv_b - cv_mid) / (cv_mid - cv_a))

    return np.mean(ratios)

In [20]:
calculate_direct_af(df['Volume'].values, df['Signal'].values)

0.9910394868602568

In [ ]:

def calculate_and_plot_direct_af(df, batch_number):
    # Filter the data for the specified batch
    batch_data = df[df['Batch'] == f'Batch_{batch_number}']
    
    # Sort the data by Volume
    batch_data = batch_data.sort_values('Volume')
    
    # Extract volume and signal data
    volume = batch_data['Volume'].values
    signal = batch_data['Signal'].values
    
    # Normalize the signal
    normalized_signal = normalize_signal(signal)
    
    # Create interpolation function
    interp_func = interp1d(normalized_signal, volume, kind='linear')
    
    # Define thresholds
    thresholds_low = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
    thresholds_high = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70]
    
    # Calculate cv_mid
    cv_mid = interp_func(0.5)
    
    # Calculate ratios and Direct AF
    ratios = []
    for low, high in zip(thresholds_low, thresholds_high):
        cv_a = interp_func(low)
        cv_b = interp_func(high)
        ratios.append((cv_b - cv_mid) / (cv_mid - cv_a))
    
    direct_af = np.mean(ratios)
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Plot the normalized signal
    sns.lineplot(x=volume, y=normalized_signal, label='Normalized Signal')
    
    # Highlight the mid-point
    plt.scatter(cv_mid, 0.5, color='red', s=100, zorder=5, label='Mid-point')
    
    # Add vertical line at mid-point
    plt.axvline(x=cv_mid, color='red', linestyle='--', alpha=0.7, label='Mid-line')
    
    # Highlight the threshold points and add ratio values
    colors = plt.cm.rainbow(np.linspace(0, 1, len(thresholds_low)))
    for i, (low, high, ratio) in enumerate(zip(thresholds_low, thresholds_high, ratios)):
        cv_a = interp_func(low)
        cv_b = interp_func(high)
        plt.scatter([cv_a, cv_b], [low, high], color=colors[i], s=50, zorder=5)
        plt.plot([cv_a, cv_mid, cv_b], [low, 0.5, high], color=colors[i], linestyle=':', alpha=0.7)
        
        # Add ratio value to the plot
        mid_x = (cv_a + cv_b) / 2
        mid_y = (low + high) / 2
        plt.annotate(f'{ratio:.2f}', (mid_x, mid_y), color=colors[i], 
                     xytext=(5, 5), textcoords='offset points', 
                     bbox=dict(boxstyle="round,pad=0.3", fc="white", ec=colors[i], alpha=0.8),
                     ha='left', va='bottom')
    
    plt.title(f'Normalized Signal vs Volume for Batch {batch_number} with Direct AF Visualization')
    plt.xlabel('Volume')
    plt.ylabel('Normalized Signal')
    plt.legend()
    
    # Add text annotation for Direct AF value
    plt.text(0.95, 0.05, f'Direct AF: {direct_af:.2f}', 
             horizontalalignment='right', verticalalignment='bottom',
             transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', edgecolor='black', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return direct_af

# Example usage:
direct_af = calculate_and_plot_direct_af(df_final, 1)  # Plot for Batch 1
print(f"Calculated Direct AF: {direct_af:.2f}")